In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical  
from keras.datasets import cifar100
import numpy as np

# (training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))

# centre the data
def centre_data(train, test):
    
    # calculate the means for each attribute of the training data
    column_means = np.mean(train, axis=0) 
    
    # centre training data by subtracting training data attribute means
    for i in range(len(train)):
        train[i] = train[i] - column_means
    
    # centre testing data by subtracting training data attribute means
    for x in range(len(test)):
        test[x] = test[x] - column_means
        
    return train, test

# apply PCA on the data 
def PCA(variance_target, training_data, testing_data):

    U, sigma, Vt = np.linalg.svd(training_data, full_matrices=False)
    
    sum_square_singular = np.sum(sigma**2)
    
    ratios = sigma**2/sum_square_singular
    
                
    n_components = 0
    explained_variance = 0
    
    # determine how many principle components must be retained to maintain the target level of explained variance
    for i in range(len(ratios)):
        if explained_variance >= variance_target:
            break
        else: 
            n_components += 1
            explained_variance += ratios[i]
    
    return training_data.dot(Vt.T[:, :n_components]), testing_data.dot(Vt.T[:, :n_components])

def load_in_dataset_and_preprocess(explained_variance):
    (training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))
    
    # reshape the data 
    training_data = training_data.reshape(50000, 3072)
    testing_data = testing_data.reshape(10000, 3072)

    # preprocess data
    validation_data = training_data[49000:, :]
    validation_labels = np.squeeze(training_labels[49000:, :])
    training_data = training_data[:49000, :]
    training_labels = np.squeeze(training_labels[:49000, :])
    
    training_data = training_data.astype('float32')
    testing_data = testing_data.astype('float32')
    validation_data = validation_data.astype('float32')

    # Centre data
    training_data, testing_data = centre_data(training_data, testing_data)

    # Apply PCA
    training_data, testing_data = PCA(explained_variance, training_data, testing_data)
    
    return training_data, training_labels, testing_data, testing_labels

training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)


In [7]:


# training_labels = to_categorical(training_labels, 20)
# testing_labels = to_categorical(testing_labels, 20)

# Normalization of pixel values (to [0-1] range)
training_data = training_data / 255
testing_data = testing_data / 255

In [22]:
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Dropout

def run_MLP_model(training_data, training_labels, testing_data, testing_labels, first_activation_function, second_activation_function, num_hidden_layers, learning_rate, optimiser, decay_level, momentum, epochs, loss):

    training_data = training_data.reshape(50000, 3072)
    testing_data = testing_data.reshape(10000, 3072)
    
    training_data = training_data.astype('float32')
    testing_data = testing_data.astype('float32')
    
    # Centre data
    training_data, testing_data = centre_data(training_data, testing_data)

    # Apply PCA
    training_data, testing_data = PCA(explained_variance, training_data, testing_data)
    
    model = Sequential()

    model.add(Dense(num_hidden_layers, activation='relu', input_dim=training_data.shape[1]))
    model.add(Dropout(0.5))
    model.add(Dense(num_hidden_layers, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(20, activation='softmax'))
    
    if optimiser == 'SGD':
        optimiser = SGD(lr=learning_rate, decay=decay_level, momentum=momentum, nesterov=True)
    else:
        optimiser = Adam(lr=0.001, decay=1e-6)

    # can also use loss function categorical_crossentropy
    # sparse_categorical_crossentropy
    # or optimiser SGD
    # try with different optimisers and loss functions
    model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])


    history = model.fit(training_data, training_labels, epochs=epochs, batch_size=32, verbose=0, validation_split=0.2)
    score = model.evaluate(testing_data, testing_labels, batch_size=128, verbose=0)

    print(score)
    return score 

In [23]:
(training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))

run_model(training_data, training_labels, testing_data, testing_labels, 'relu', 'softmax', 256, 0.001, 'Adam', 1e-6, 0.9, 20, 'sparse_categorical_crossentropy')


['loss', 'accuracy']
[2.9958221912384033, 0.05000000074505806]


In [ ]:
### Converting images to grayscale

In [ ]:
def convert_to_grayscale(train, test):
    rgb_weights = [0.2989, 0.5870, 0.1140]
    gray_train = np.dot(train[:][...,:3], rgb_weights)
    gray_test = np.dot(test[:][...,:3], rgb_weights)
    return gray_train.astype(int), gray_test.astype(int)

In [ ]:
### Increasing image contrast

In [ ]:
def increase_image_contrast(image, strength):
    p_low, p_high = np.percentile(image, (strength, 100 - strength))
    return (exposure.rescale_intensity(image, in_range=(p_low, p_high))).astype(int)

def increase_all_contrast(train, test, strength=10):
    new_train = np.copy(train)
    new_test = np.copy(test)
    for i in range(len(train)):
        new_train[i] = increase_image_contrast(train[i], strength)
    for i in range(len(test)):
        new_test[i] = increase_image_contrast(test[i], strength)
    return new_train, new_test

In [ ]:
### Image equalisation

In [ ]:
def equalize_hist(image):
    return (exposure.equalize_hist(image) * 255).astype(int)

def equalize_hist_all(train, test):
    new_train = np.copy(train)
    new_test = np.copy(test)
    for i in range(len(train)):
        new_train[i] = equalize_hist(train[i])
    for i in range(len(test)):
        new_test[i] = equalize_hist(test[i])
    return new_train, new_test



In [ ]:
def equalize_adapthist(image, clip_lim):
    return (exposure.equalize_adapthist(image, clip_limit=clip_lim) * 255).astype(int)

def equalize_adapthist_all(train, test, clip_limit=0.03):
    new_train = np.copy(train)
    new_test = np.copy(test)
    for i in range(len(train)):
        new_train[i] = equalize_adapthist(train[i], clip_limit)
    for i in range(len(test)):
        new_test[i] = equalize_adapthist(test[i], clip_limit)
    return new_train, new_test

In [ ]:
### Edge detection

In [ ]:
def canny_edge_filter_all(train, test, sig=1):
    if (len(train.shape) > 3):
        train, test = convert_to_grayscale(train, test)
    new_train = np.copy(train)
    new_test = np.copy(test)
    for i in range(len(train)):
        new_train[i] = feature.canny(train[i].astype(float), sigma=sig)
    for i in range(len(test)):
        new_test[i] = feature.canny(test[i].astype(float), sigma=sig)
    return new_train, new_test

In [ ]:
### flattening the images

In [ ]:
def flatten_data(x_train, y_train, x_test, y_test):
    new_image_shape = 1
    for dim in range(1, len(x_train.shape)):
        new_image_shape *= x_train.shape[dim]
        
    flat_x_train = x_train.reshape((x_train.shape[0], new_image_shape))
    flat_y_train = np.ravel(y_train)
    
    flat_x_test = x_test.reshape((x_test.shape[0], new_image_shape))
    flat_y_test = np.ravel(y_test)
    return flat_x_train, flat_y_train, flat_x_test, flat_y_test

In [ ]:
### Testing

In [ ]:
def import_data() -> ((np.ndarray, np.ndarray), (np.ndarray, np.ndarray)):
    return tf.keras.datasets.cifar100.load_data(label_mode="coarse")

In [ ]:
import time
import csv

In [ ]:
def nano_to_seconds(nanoseconds):
    """Converts nanoseconds to seconds rounded to the nearest 5 decimal places.
    
    Parameters
    ----------
    nanoseconds : int
        The nanoseconds to convert
    """

    return np.round((nanoseconds / 1e+9), 5)

In [ ]:
## test preprocessing

In [ ]:
## test optimiser 

In [ ]:
## test second activation functions

In [ ]:
def test_MLP_second_activation_functions():
    activation_functions = ['logistic', 'tanh', 'softmax']
    (raw_x_train, raw_y_train), (raw_x_test, raw_y_test) = import_data()
    with open('rf_estimators.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Layers', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
        optimiser = 'Adam'
        epochs = 50
        num_hidden_layers = 256
        for activation_function in activation_functions:
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, testing_data, testing_labels, 'relu', activation_function, 256, 0.001, 'Adam', 1e-6, 0.9, 20, 'sparse_categorical_crossentropy')
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, second_activation_function, loss, learning_rate, optimiser, momentum, decay, epochs, avg_accuracy, avg_runtime])
            print("Finished n_estimators: {}".format(n_estimators))

In [ ]:
## test first activation functions

In [ ]:
def test_MLP_second_activation_functions():
    activation_functions = []
    (raw_x_train, raw_y_train), (raw_x_test, raw_y_test) = import_data()
    with open('rf_estimators.csv', mode='w', newline='') as csv_file:
        result_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(['1st Activation Function', '2nd Activation Function', 'Loss Function', 'Learning Rate', 'Optimiser', 'Momentum', 'Decay', 'Epochs', 'Number Hidden Layers', 'Average Runtime'])
        momentum = 0.9
        decay=1e-6
        learning_rate = 0.001
        first_activation_function = 'relu'
        second_activation_function = 'softmax'
        loss = 'sparse_categorical_crossentropy'
        optimiser = 'Adam'
        epochs = 50
        num_hidden_layers = 256
        for activation_function in activation_functions:
            num_tests = 0
            total_accuracy = 0
            total_runtime = 0
            for i in range(2):
                start_time = time.time_ns()
                total_accuracy += run_MLP_model(training_data, training_labels, testing_data, testing_labels, 'relu', activation_function, 256, 0.001, 'Adam', 1e-6, 0.9, 20, 'sparse_categorical_crossentropy')
                end_time = time.time_ns()
                total_runtime += nano_to_seconds(end_time - start_time)
                num_tests += 1
            avg_accuracy = np.round(total_accuracy / float(num_tests), 5)
            avg_runtime = np.round(total_runtime / float(num_tests), 5)
            result_writer.writerow([first_activation_function, second_activation_function, loss, learning_rate, optimiser, momentum, decay, epochs, avg_accuracy, avg_runtime])
            print("Finished n_estimators: {}".format(n_estimators))

In [ ]:
## test learning rates

In [ ]:
## test number of hidden layers

In [ ]:
## test decay

In [ ]:
## test momentum 

In [ ]:
## test epochs